In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
import seaborn as sns

In [ ]:
data=pd.read_csv("/kaggle/input/cassava-leaf-disease-classification/train.csv")

In [ ]:
data.shape

In [ ]:
data.head()


In [ ]:
data['label'].value_counts()

In [ ]:
data['label'].hist()

In [ ]:
import json
file= open('/kaggle/input/cassava-leaf-disease-classification/label_num_to_disease_map.json')
read_labels=json.load(file)
read_labels={int(label):disease_name for label,disease_name in read_labels.items()}

In [ ]:
data['disease_name']=data.label.map(read_labels)
data.head()

In [ ]:
data['disease_name'].value_counts()

In [ ]:
data.drop('label',axis=1,inplace=True)

In [ ]:
data.head()

In [ ]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten
from keras.layers import Conv2D,MaxPooling2D,Activation,AveragePooling2D,BatchNormalization
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split = 0.2
)

valid_datagen = ImageDataGenerator(
    rescale=1/255,
    validation_split = 0.2
)

In [ ]:
images_dir_path = "/kaggle/input/cassava-leaf-disease-classification/train_images"

In [ ]:
img_width,img_height =256,256
input_shape=(img_width,img_height,3)

In [ ]:
batch_size =32
train_generator = train_datagen.flow_from_dataframe(
    dataframe=data,
    directory = images_dir_path,
    x_col = "image_id",
    y_col = "disease_name",
    target_size = (img_width,img_height),
    class_mode = "categorical",
    batch_size = batch_size
)

valid_generator = valid_datagen.flow_from_dataframe(
    dataframe = data,
    directory = images_dir_path,
    x_col = "image_id",
    y_col = "disease_name",
    target_size = (img_width,img_height),
    class_mode = "categorical",
    batch_size = batch_size
)



In [ ]:
model = Sequential()
model.add(Conv2D(32, (5, 5),input_shape=input_shape,activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Conv2D(32, (3, 3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))   
model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(128,activation='relu'))          
model.add(Dense(5,activation='softmax'))

In [ ]:
import keras
opt=keras.optimizers.Adam(lr=0.001)
model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])
train=model.fit_generator(train_generator,
                          epochs=15,
                          steps_per_epoch=train_generator.samples // batch_size,
                          validation_data=valid_generator)

In [ ]:
#Plottinf the accuracy
acc = train.history['accuracy']
val_acc = train.history['val_accuracy']
loss = train.history['loss']
val_loss = train.history['val_loss']
epochs = range(1, len(acc) + 1)

In [ ]:
#Train and validation accuracy
plt.plot(epochs, acc, 'b', label='Training accurarcy')
plt.plot(epochs, val_acc, 'r', label='Validation accurarcy')
plt.title('Training and Validation accurarcy')
plt.legend()
plt.figure()


In [ ]:
#Evaluating the model for unseen data
score,accuracy =model.evaluate(valid_generator,verbose=1)
print("Test score is {}".format(score))
print("Test accuracy is {}".format(accuracy))

In [ ]:
# Save entire model with optimizer, architecture, weights and training configuration.
from keras.models import load_model
model.save('model.h5')

# Save model weights.
from keras.models import load_model
model.save_weights('model_weights.h5')